# Ensambles #

In [106]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate


from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from mlens.ensemble import SuperLearner 

from itertools import combinations
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import MaxAbsScaler
from sklearn.kernel_approximation import RBFSampler
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.kernel_approximation import Nystroem

from sklearn.neural_network import MLPClassifier

In [2]:
tf = pd.read_csv('featuresMagui.csv') #mis features 
tf_seba = pd.read_csv('features_seba.csv') #los features de Seba
tf_santi = pd.read_csv('santi_timefeatures.csv')
tf_santi_2 = pd.read_csv('Santi_FeaturesConEventos.csv')
#tf_2 = pd.read_csv('featuresMagui2.csv')
labels = pd.read_csv('../data/labels_training_set.csv') #las personas de las cuales tengo Info
personas =pd.read_csv('../data/trocafone_kaggle_test.csv') #las personas a las que le tengo que predecir

In [3]:
#Datos a entregar
datos = pd.merge(tf, tf_seba, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_santi, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_santi_2, on = 'person', how = 'inner')
#datos = pd.merge(datos, tf_2, on = 'person', how = 'inner')
datos = pd.merge(personas, datos, on = 'person', how = 'inner')

In [4]:
#Datos para entrenar
labels_f = pd.merge(tf,tf_seba, on = 'person',how = 'inner')
labels_f = pd.merge(labels_f,tf_santi, on = 'person',how = 'inner')
labels_f = pd.merge(labels_f,tf_santi_2, on = 'person',how = 'inner')
#labels_f = pd.merge(labels_f,tf_2, on = 'person',how = 'inner')
labels_f = pd.merge(labels, labels_f, on = 'person', how = 'inner')

In [5]:
#labels_f.columns

In [94]:
#labels_f  = labels_f.drop('viewed product', axis = 1)
#print('viewed product' in labels_f.columns.tolist())
y = labels_f.iloc[:,1:2]
X = labels_f.iloc[:,3: 1314]

In [95]:
X = X.drop('device_type', axis = 1)
print('viewed product' in X.columns.tolist())

True


In [96]:
 atributosImportantes = ['cantidadDeVisitas','FacebookAds','deviceSmartphone','cantVisitasAVP','noche','cantDirect','cantReferral','cantVisitasDomingo','cantVisitasMiercoles','cantidadDeEventos','cantBandListing','cantCheckout','cantStaticpage','cantViewedProduct','cantVisitedSite','checkout_x','searchEngineHit','brand listing mes 5','checkout mes 5','viewed product mes 5','visited site mes 5','dias_hasta_ultimo','ad campaign hit mes 4','brand listing mes 4','generic listing mes 4','search engine hit mes 4','viewed product mes 4','visited site mes 4','distan_dias','nuevo_mes_5','cant_dias_dist','modelos_dist','promedio_por_mes','mes_primer_entrada','promedio_por_dia_y','mes_3','mes_4','mes_5','dia_2','dia_3','dia_5','dia_7','dia_8','dia_9','dia_25','dia_28','dia_30','dia_31','primer_quincena','segunda_quincena','Friday','Thursday','Tuesday','fin_de_semana','martes_miercoles_jueves','hora_10','hora_12','hora_13','hora_15','hora_19','hora_rango_4-7','hora_rango_12-15','hora_rango_20-23','Afternoon','Evening','minuto_0-19','minuto_20-39','minuto_40-59','semana_del_anio_20','semana_del_anio_22','semana_del_anio_17','semana_del_anio_15','semana_del_anio_14','dia_del_anio_151','Unnamed: 0','ad campaign hit','brand listing','checkout_y','conversion','generic listing','staticpage_y','viewed product','visited site','vis_Samsung Galaxy S7 Edge','vis_Samsung Galaxy S6 Flat','vis_Samsung Galaxy S6 Edge','vis_Samsung Galaxy A7 2017','viewedproduct_Preto','viewedproduct_Branco','viewedproduct_Prateado','viewedproduct_Prata','viewedproduct_Rosa','viewedproduct_Cinza','viewedproduct_Verde','viewedproduct_Vermelho','viewedproduct_16GB','viewedproduct_32GB','viewedproduct_8GB','viewedproduct_Bom','viewedproduct_Muito Bom','visitedsite_Computer','visitedsite_Smartphone', 'visitedsite_320x570','visitedsite_1920x1080','visitedsite_Chrome 64.0']

In [97]:
X = X.loc[:,atributosImportantes]

In [98]:
scaler = preprocessing.MaxAbsScaler().fit(X)
X_transformed = scaler.transform(X)

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

In [100]:
rus= RandomUnderSampler(return_indices=True)

#id_rus son los índices
X_train, y_train, id_rus = rus.fit_sample(X_train, y_train)

In [101]:
def aplicarModelo(modelo,X_train, X_test, y_train, y_test):
    modelo.fit(X_train, y_train)#, early_stopping_rounds = 5, eval_set=[(X_test, y_test)])
    preds = modelo.predict(X_test)
    preds_prob = modelo.predict_proba(X_test)[:,1]
    train_accuracy = accuracy_score(y_train, modelo.predict(X_train))
    test_accuracy = accuracy_score(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    print('train acurracy: ')
    print(train_accuracy)
    print('test acurracy: ')
    print(test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ')
    print(area_debajo_de_curva)

Pruebo con un Ensamble: Bagging

In [102]:
xg_reg = xgb.XGBClassifier(learning_rate =0.05,n_estimators=100,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)

bag_class =BaggingClassifier(xg_reg)

aplicarModelo(bag_class,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/bagging.py:621: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.8927765237020316
test acurracy: 
0.7590113285272915
Matriz de confusión: 
[[1401  447]
 [  21   73]]
Área bajo la curva: 
0.8551510546191399


Pruebo con una red neuronal.

In [15]:
nn = MLPClassifier( early_stopping=False, random_state=123)

aplicarModelo(nn,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.9695259593679458
test acurracy: 
0.7435633367662204
Matriz de confusión: 
[[1374  474]
 [  24   70]]
Área bajo la curva: 
0.8070254213871235


/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Pruebo con una red neuronal pero antes aumento las dimensiones con un Kernel para que pueda hacerse una separación lineal (the kernel Trick).

In [86]:
 atributosImportantes = ['cantidadDeVisitas','FacebookAds','deviceSmartphone','cantVisitasAVP','noche','cantDirect','cantReferral','cantVisitasDomingo','cantVisitasMiercoles','cantidadDeEventos','cantBandListing','cantCheckout','cantStaticpage','cantViewedProduct','cantVisitedSite','checkout_x','searchEngineHit','brand listing mes 5','checkout mes 5','viewed product mes 5','visited site mes 5','dias_hasta_ultimo','ad campaign hit mes 4','brand listing mes 4','generic listing mes 4','search engine hit mes 4','viewed product mes 4','visited site mes 4','distan_dias','nuevo_mes_5','cant_dias_dist','modelos_dist','promedio_por_mes','mes_primer_entrada','promedio_por_dia_y','mes_3','mes_4','mes_5','dia_2','dia_3','dia_5','dia_7','dia_8','dia_9','dia_25','dia_28','dia_30','dia_31','primer_quincena','segunda_quincena','Friday','Thursday','Tuesday','fin_de_semana','martes_miercoles_jueves','hora_10','hora_12','hora_13','hora_15','hora_19','hora_rango_4-7','hora_rango_12-15','hora_rango_20-23','Afternoon','Evening','minuto_0-19','minuto_20-39','minuto_40-59','semana_del_anio_20','semana_del_anio_22','semana_del_anio_17','semana_del_anio_15','semana_del_anio_14','dia_del_anio_151','Unnamed: 0','ad campaign hit','brand listing','checkout_y','conversion','generic listing','staticpage_y','viewed product','visited site','vis_Samsung Galaxy S7 Edge','vis_Samsung Galaxy S6 Flat','vis_Samsung Galaxy S6 Edge','vis_Samsung Galaxy A7 2017','viewedproduct_Preto','viewedproduct_Branco','viewedproduct_Prateado','viewedproduct_Prata','viewedproduct_Rosa','viewedproduct_Cinza','viewedproduct_Verde','viewedproduct_Vermelho','viewedproduct_16GB','viewedproduct_32GB','viewedproduct_8GB','viewedproduct_Bom','viewedproduct_Muito Bom','visitedsite_Computer','visitedsite_Smartphone', 'visitedsite_320x570','visitedsite_1920x1080','visitedsite_Chrome 64.0']

In [87]:
y = labels_f.iloc[:,1:2]
X = labels_f.iloc[:,3: 1314]

In [88]:
X = X.loc[:,atributosImportantes]

In [89]:
scaler = preprocessing.MaxAbsScaler().fit(X)
X_transformed = scaler.transform(X)

In [90]:
rbf_feature = RBFSampler(gamma=3,n_components= 100, random_state=123)
X_features = rbf_feature.fit_transform(X)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size= 0.1, random_state= 123)

In [92]:
rus= RandomUnderSampler(return_indices=True)

#id_rus son los índices
X_train, y_train, id_rus = rus.fit_sample(X_train, y_train)

In [93]:
nn = MLPClassifier( early_stopping=False, random_state=123)

aplicarModelo(nn,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
1.0
test acurracy: 
0.48094747682801237
Matriz de confusión: 
[[878 970]
 [ 38  56]]
Área bajo la curva: 
0.5386732062263978


/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Pruebo con SVM y con The Kernel trick

In [24]:
clf = svm.SVC(gamma='scale', probability = True)

aplicarModelo(clf,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.6139954853273137
test acurracy: 
0.529351184346035
Matriz de confusión: 
[[984 864]
 [ 50  44]]
Área bajo la curva: 
0.5473456065211386


Pruebo SVM con pesos y con The Kernel Trick

In [25]:
# fit the model and get the separating hyperplane using weighted classes
wclf = svm.SVC(kernel='linear', class_weight={1: 3, 0: 1}, probability = True)

aplicarModelo(wclf,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.5
test acurracy: 
0.04840370751802266
Matriz de confusión: 
[[   0 1848]
 [   0   94]]
Área bajo la curva: 
0.44125909551441467


Pruebo SVM con pesos y con The Kernel Trick (sin undersampling)

In [81]:
y = labels_f.iloc[:,1:2]
X = labels_f.iloc[:,3: 1314]

In [82]:
X = X.loc[:,atributosImportantes]

In [83]:
scaler = preprocessing.MaxAbsScaler().fit(X)
X_transformed = scaler.transform(X)

In [84]:
rbf_feature = RBFSampler(gamma=3,n_components= 100, random_state=123)
X_features = rbf_feature.fit_transform(X)

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size= 0.1, random_state= 123)

In [31]:
wclf = svm.SVC(kernel='linear', class_weight={1: 7, 0: 1}, probability = True)

aplicarModelo(wclf,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.949290293040293
test acurracy: 
0.9515962924819773
Matriz de confusión: 
[[1848    0]
 [  94    0]]
Área bajo la curva: 
0.5470894353873077


Reduzco dimensiones (PCA) y aplico KNN. Esto se debe a que KNN sufre la maldición de la dimensionalidad. 

In [105]:
from sklearn.decomposition import PCA

scaler = preprocessing.MaxAbsScaler().fit(X)
X_transformed = scaler.transform(X)

pca = PCA(n_components=60).fit(X)
X_reduce = pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_reduce, y, test_size= 0.1, random_state= 123)



rus= RandomUnderSampler(return_indices=True)

#id_rus son los índices
X_train, y_train, id_rus = rus.fit_sample(X_train, y_train)


In [104]:
knn = KNeighborsClassifier(n_neighbors= 500)

aplicarModelo(knn, X_train, X_test, y_train, y_test)

train acurracy: 
0.9485929794023789
test acurracy: 
0.9515962924819773
Matriz de confusión: 
[[1848    0]
 [  94    0]]
Área bajo la curva: 
0.5012002625034541


Aplico Bagging a KNN

In [39]:
bag_class =BaggingClassifier(knn)
aplicarModelo(bag_class,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/bagging.py:621: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.7618510158013544
test acurracy: 
0.8007209062821833
Matriz de confusión: 
[[1503  345]
 [  42   52]]
Área bajo la curva: 
0.8025755273095698


Aplico RandomForest y PCA

In [56]:
clf = RandomForestClassifier(n_estimators=60, max_depth=6, random_state=0)
aplicarModelo(clf,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


train acurracy: 
0.8826185101580135
test acurracy: 
0.7239958805355304
Matriz de confusión: 
[[1329  519]
 [  17   77]]
Área bajo la curva: 
0.8237369899603944


Aplico bagging y random Forest

In [42]:
bag_class =BaggingClassifier(clf)
aplicarModelo(bag_class,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/bagging.py:621: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.8594808126410836
test acurracy: 
0.7106076210092688
Matriz de confusión: 
[[1307  541]
 [  21   73]]
Área bajo la curva: 
0.8255100396057843


xgboost y PCA

In [57]:
xg_reg = xgb.XGBClassifier(learning_rate =0.05,n_estimators=100,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
aplicarModelo(xg_reg,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.9514672686230248
test acurracy: 
0.7296601441812565
Matriz de confusión: 
[[1340  508]
 [  17   77]]
Área bajo la curva: 
0.8173471032513585


In [60]:
bag_class =BaggingClassifier(xg_reg,max_samples = 1000)
aplicarModelo(bag_class,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/bagging.py:621: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.8809255079006773
test acurracy: 
0.721936148300721
Matriz de confusión: 
[[1323  525]
 [  15   79]]
Área bajo la curva: 
0.8280890209081697


Hago un ensamble de Xgboost usando super learner (este lo sacao pq me dio mal)

In [65]:
xg_reg = xgb.XGBClassifier(learning_rate =0.05,n_estimators=10,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
xg_reg_2 = xgb.XGBClassifier(learning_rate =0.01,n_estimators=10,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
xg_reg_3 = xgb.XGBClassifier(learning_rate =0.1,n_estimators=100,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
#gaussiano =GaussianNB()

#clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

#knn = KNeighborsClassifier(n_neighbors=4)

ensemble = SuperLearner(scorer = accuracy_score, random_state = 123, folds = 10)
#ensemble.add(gaussiano)
#ensemble.add(clf)
#ensemble.add(knn)
ensemble.add(xg_reg_2)
ensemble.add(xg_reg_3)
ensemble.add_meta(xg_reg)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.9536973443223443
test acurracy: 
0.952626158599382
Matriz de confusión: 
[[1847    1]
 [  91    3]]
Área bajo la curva: 
0.5156868840379479
